In [1]:
# example string to do mini tests on
example = "I just EASILY had the BEST lunch I've ever eaten!  It was THAT good!\n\nThe chicken tortilla soup was out of this world...light and delicate...fresh and HOT!!!\nI had two fish tacos with no tortilla.  One was a regular fish taco and the other was a beer battered fish taco.l\n\nThe fire roasted salsa was EASILY the best salsa I have ever had, too!\n\nThis place is a serious gem!  I could go there every single day!\n\nThanks guys!!"

# Import Benepar


In [2]:
import benepar
benepar.download("benepar_en2")
parser = benepar.Parser("benepar_en2")

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

# Import StanfordNLP

Remember to first instantiate stanfordNLP in java!

java command: 

``
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -annotators "tokenize,ssplit,pos,lemma,parse,sentiment" -sentiment.threads 8 -port 9000 -timeout 30000``


In [3]:
import stanfordnlp
from stanfordcorenlp import StanfordCoreNLP
import logging
import json

class StanfordNLP:
    def __init__(self, host='http://localhost', port=9000):
        self.nlp = StanfordCoreNLP(host, port=port,
                                   timeout=30000)  # , quiet=False, logging_level=logging.DEBUG)
        self.props = {
            'annotators': 'tokenize,ssplit,pos,lemma,ner,parse,depparse,dcoref,relation',
            'pipelineLanguage': 'en',
            'outputFormat': 'json'
        }

    def word_tokenize(self, sentence):
        return self.nlp.word_tokenize(sentence)

    def pos(self, sentence):
        return self.nlp.pos_tag(sentence)

    def ner(self, sentence):
        return self.nlp.ner(sentence)

    def parse(self, sentence):
        return self.nlp.parse(sentence)

    def dependency_parse(self, sentence):
        return self.nlp.dependency_parse(sentence)

    def annotate(self, sentence):
        return json.loads(self.nlp.annotate(sentence, properties=self.props))

    @staticmethod
    def tokens_to_dict(_tokens):
        tokens = defaultdict(dict)
        for token in _tokens:
            tokens[int(token['index'])] = {
                'word': token['word'],
                'lemma': token['lemma'],
                'pos': token['pos'],
                'ner': token['ner']
            }
        return tokens

    

sNLP = StanfordNLP()
text = 'A blog post using Stanford CoreNLP Server. Visit www.khalidalnajjar.com for more details.'
print("Parse:", sNLP.parse(text))

Parse: (ROOT
  (S
    (NP (DT A) (NN blog))
    (VP (NN post)
      (S
        (VP (VBG using)
          (NP (NNP Stanford) (NNP CoreNLP) (NN Server)))))
    (. .)))


# Import Entity Recognition Model

In [6]:
import spacy

model_dir = "./models"
nlp = spacy.load(model_dir)

In [7]:
def get_entities(nlp_model, text):
    """
    Input nlp_model and text, retrieve a list of unique entities from the text.
    """
    doc = nlp_model(text)
    entities = set()
    for ent in doc.ents:
        if ent.label_ == "PRODUCT":
            entities.add(ent.text)
    return list(entities)

In [8]:
example_entity_list = get_entities(nlp, example)
example_entity_list

['soup',
 'chicken',
 'fish',
 'taco',
 'place',
 'lunch',
 'gem',
 'beer',
 'tortilla',
 'salsa']

In [9]:
#quick fix
#example_entity_list = ['chicken tortilla soup', 'fish tacos', 'tortilla', 'beer battered fish taco', 'fire roasted salasa', 'place']

In [10]:
example

"I just EASILY had the BEST lunch I've ever eaten!  It was THAT good!\n\nThe chicken tortilla soup was out of this world...light and delicate...fresh and HOT!!!\nI had two fish tacos with no tortilla.  One was a regular fish taco and the other was a beer battered fish taco.l\n\nThe fire roasted salsa was EASILY the best salsa I have ever had, too!\n\nThis place is a serious gem!  I could go there every single day!\n\nThanks guys!!"

# Sentiment Helper Code

In [11]:
import collections
import re

def remove_nestings(lst): 
    output = []
    
    def remove_nestings_recursive(l):
        for i in l: 
            if type(i) == list: 
                remove_nestings_recursive(i) 
            else: 
                output.append(i)
    
    remove_nestings_recursive(lst)
    
    return output

def continue_splitting(review,list_of_dividers):
        
    temp = list_of_dividers.copy()
    l = [review]
    while len(temp) > 0:
        divider = temp.pop(0)
        l_new = []
        for i in l:
            l_new += i.split(divider)
        l = l_new
    return l

def join_clause(review, list_of_split_clauses, list_of_dividers):
    output = []
    loc_of_split_clauses = []
    for clause in list_of_split_clauses:
        loc_of_split_clauses.append(review.find(clause))
    for divider in list_of_dividers:
        print(divider)
        loc_div = review.find(divider)
        print(loc_div)
        for i in range(len(loc_of_split_clauses)):
            if loc_div > loc_of_split_clauses[i]:
                print(loc_div,loc_of_split_clauses[i])
                
def join_partitions(long_review,entity_with_review):
    loclist = []
    for (_, clause) in entity_with_review:
        loclist.append((long_review.find(clause),long_review.find(clause)+len(clause)))
    starts = {i for (i,j) in loclist}
    ends = {j for (i,j) in loclist}
    starts.add(len(long_review))
    newends = {}
    for i in ends:
        newends[i] = min([x for x in starts if x >= i])
    for i in newends:
        pass
    new_entity_with_review = []
    for i in range(len(loclist)):
        tup = loclist[i]
        entity = entity_with_review[i][0]
        st = tup[0]
        en = newends[tup[1]]
        new_entity_with_review.append((entity,long_review[st:en]))
    return new_entity_with_review

def split_long_string(review):
    num = len(review)
    split_list = []
    start = 0
    end = 0
    while num != end:
        if num - end < 1000:
            end = num
            split_list.append(review[start:end])
        else:
            end = review[start:(start+1000)].rfind('.')
            if end == -1:
                end = review[start:(start+1000)].rfind(' ') #if no '.', space will do
                if end == -1:
                    end = min(start + 1000,num)
            split_list.append(review[start:end])
            start = end
    print(start,end)
    return(split_list)

def split_review_naive(review,entities):
    clauses = re.split('[.?!]',review)
    lenlist = [len(x) for x in clauses]
    clauses = [x for _, x in sorted(zip(lenlist,clauses),reverse=False)]
    entity_with_clause = []
    for entity in entities:
        for clause in clauses:
            if entity in clause:
                entity_with_clause.append((entity,clause))
                break
    return(join_partitions(review,entity_with_clause))

def min_tree(review, entitiess, parser, output = 'minimum'):
    
    #review is string, entities is list of strings, parser is parser object
    # TODO: How well are each review punctuatd and so forth EDA
    if output == 'partition':
        full_review = ''
        
    treelist = []
    lenlist = []
    temp = review.split('\n')
    
    if output == 'no_parse':
        return(split_review_naive(review,entities))
    
    if len(review) > 1000:
        split_reviews = split_long_string(review)
    else:
        split_reviews = [i for i in temp if len(i) > 1 and len(i) <= 1000 ]
    
    for rev in split_reviews:
        if rev and rev.strip():
            u = parser.parse(rev) # tree 

            if type(u) == str:
                u = nltk.Tree.fromstring(u)

            for s in u.subtrees(): # subtrees 
                if s.label() == 'S': # if sentence
                    treelist += [s]
                    lenlist += [len(s.leaves())] # how long clause
                        
            if output == 'partition':
                full_review += ' '.join(u.leaves())

    treelist = [x for _, x in sorted(zip(lenlist,treelist),reverse=False)] # sort by lenlisit
    clauses = [' '.join(tree.leaves()) for tree in treelist]
    if not clauses:
        clauses.append(review)
    entity_with_clause = []
    
    if output == 'all':
        for entity in entities:
            clauselist = []
            for clause in clauses:
                if entity in clause:
                    clauselist.append(clause)
            entity_with_clause.append((entity,clauselist))
    
    #TODO: create rules and test them
    elif output == 'minimum':
        for entity in entities:
            for clause in clauses:
                if entity in clause:
                    entity_with_clause.append((entity,clause))
                    break
                    
    elif output == 'partition':
        #first find minimal clause
        for entity in entities:
            for clause in clauses:
                if entity in clause:
                    entity_with_clause.append((entity,clause))
                    break
        #get location of minimal clause in review
        
        
        entity_with_clause = join_partitions(full_review,entity_with_clause)
    
    return entity_with_clause

# Code to Perform Sentiment Analysis

In [19]:
#STANFORD NLP
import numpy as np
from pycorenlp import StanfordCoreNLP
def stanford_sentiment_start():
    nlp = StanfordCoreNLP('http://localhost:9000')
    return nlp

def stanford_sentiment(entity_with_clause):
    nlp = stanford_sentiment_start()
    entity_with_sentiment = []
    for entity, clause in entity_with_clause:
        result = nlp.annotate(clause,
                   properties={
                       'annotators': 'sentiment',
                       'outputFormat': 'json'
                   })
        sentiment = np.dot(result['sentences'][0]['sentimentDistribution'], [-2, -1, 0, 1, 2])
        entity_with_sentiment.append((entity, sentiment))
    return entity_with_sentiment

#VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def vader_sentiment(entity_with_clause):
    analyzer = SentimentIntensityAnalyzer()
    entity_with_sentiment = []
    for entity, clause in entity_with_clause:
        sentiment = analyzer.polarity_scores(clause)['compound']
        entity_with_sentiment.append((entity,sentiment))
    return(entity_with_sentiment)       

# Werk

In [20]:
def werk(review, entities, parser = [], sentiment_package = 'vader', parse_package = 'benepar', rule = 'rule_1'):
    
    #print("\nLoading Parser")
    
    #first is the parser
    if not parser and parse_package == 'benepar':
        parser = benepar.Parser("benepar_en2")
    elif not parser and parse_package == 'stanford':
        #parser = StanfordNLP('http://localhost')
        raise Exception('incorrect parse package')
    elif parser:
        pass
    else:
        raise Exception('incorrect parse package')
    
    #print("Parser =", parse_package)

        
    #second is the rule
    
    #print("\nLoading Rule")
    
    if rule == 'rule_1':
        
        #print("Rule =",rule)
        
        entity_with_review = min_tree(review, entities, parser, output = 'minimum')
        
        #print("\nTree Generated")
        
        entity_with_sentiment = sentiment_analysis(entity_with_review, sentiment_package)
        
        #print("\nSentiment Generated")

        
    elif rule == 'rule_2':
        
        #print("Rule =",rule)
        
        entity_with_review = min_tree(review, entities, parser, output = 'all')
        
        #print("\nTree Generated")
        entity_with_sentiment = []
        sentiment = 0
        for ent, revlist in entity_with_review:
            for clause in revlist:
                sentiment = sentiment_analysis_indiv(clause,sentiment_package)
                if sentiment_package == 'benepar' and abs(sentiment) != 0:
                    break
                elif sentiment_package == 'stanford' and abs(sentiment) > 0.5:
                    break
                    #if sentiment is not neutral, stop. If sentiment is neutral, keep going up tree.
            entity_with_sentiment.append((ent,sentiment))
        #print("\nSentiment Generated")        
        
    elif rule == 'rule_3':
        
        #print("Rule =",rule)
        
        entity_with_review = min_tree(review, entities, parser, output = 'all')
        
        #print("\nTree Generated")
        
        entity_with_sentiment = []
        for ent, revlist in entity_with_review:
            sentiment_list = []
            for clause in revlist:
                sentiment = sentiment_analysis_indiv(clause,sentiment_package)
                sentiment_list.append(sentiment)
            entity_with_sentiment.append((ent,np.mean(sentiment_list)))
            
        #print("\nSentiment Generated") 
        
    elif rule == 'rule_4':
        
        #print("Rule =",rule)
        
        entity_with_review = min_tree(review, entities, parser, output = 'partition')
        #print("\nTree Generated")
        
        entity_with_sentiment = sentiment_analysis(entity_with_review, sentiment_package)
        
        #print("\nSentiment Generated")
        
    elif rule == 'rule_5':
        
        #print("Rule =",rule)
        
        entity_with_review = min_tree(review, entities, parser, output = 'minimum')
        entity_with_review_p = min_tree(review, entities, parser, output = 'partition')
        #print("\nTree Generated")
        
        entity_with_sentiment = sentiment_analysis(entity_with_review, sentiment_package)
        for i in range(len(entity_with_sentiment)):
            sent = entity_with_sentiment[i][1]
            if sentiment_package == 'vader' and sent != 0:
                entity_with_sentiment[i] = (entity_with_sentiment[i][0],sentiment_analysis_indiv(entity_with_review_p[i][1],sentiment_package))
            elif sentiment_package == 'stanford' and abs(sent) > 0.5:
                entity_with_sentiment[i] = (entity_with_sentiment[i][0],sentiment_analysis_indiv(entity_with_review_p[i][1],sentiment_package))
    
    elif rule == 'rule_6':
        
        entity_with_review = min_tree(review, entities, parser, output = 'no_parse')
        entity_with_sentiment = sentiment_analysis(entity_with_review, sentiment_package)
        
        #print("\nSentiment Generated")
    
    else:
        raise Exception('incorrect rule')
    
    return(entity_with_sentiment)
    
def sentiment_analysis(entity_with_review, sentiment_package = 'stanford'):
    #takes in list of tuples
    if sentiment_package == 'stanford':
        return stanford_sentiment(entity_with_review)
    elif sentiment_package == 'vader':
        return vader_sentiment(entity_with_review)
    else:
        raise Exception('incorrect sentiment package')

def sentiment_analysis_indiv(clause,sentiment_package = 'stanford'):
    #takes in a single review
    if sentiment_package == 'stanford':
        nlp = stanford_sentiment_start()
        result = nlp.annotate(clause,
                   properties={
                       'annotators': 'sentiment',
                       'outputFormat': 'json'
                   })
        print(result['sentences'][0]['sentimentDistribution'])
        return np.dot(result['sentences'][0]['sentimentDistribution'], [-2, -1, 0, 1, 2])
    elif sentiment_package == 'vader':
        analyzer = SentimentIntensityAnalyzer()
        return analyzer.polarity_scores(clause)['compound']
    else:
        raise Exception('incorrect sentiment package')

# Perform End-to-End Validation

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
from tqdm.notebook import tqdm
from collections import defaultdict

In [22]:
bus = pd.read_csv("bus_same_stars_and_reviews.csv")

In [23]:
business_ids_similar_stars = bus.business_id.unique()

In [24]:
from scipy.stats import spearmanr
rules = ['rule_1', 'rule_2', 'rule_3', 'rule_4', 'rule_5', 'rule_6']

rule = 'rule_1'

correlation_scores = []

print("RULE: ", rule)
for bus_id in tqdm(business_ids_similar_stars):
    print("Running on restaurant ", bus_id, "...")
    subset = bus[bus.business_id == bus_id]
    
    # only get reviews with enough amount of text
    reviews_subset = [review for review in subset.text if len(review) < 400]

    print("Number of Reviews left after subset length: ", len(reviews_subset))
    
    # get set of entities for this particular restaurant,
    # and count how many reviews each entity have
    entities_with_count = defaultdict(int) 
    review_entities = [] # extract entities for each review
    print("Extracting entities from each review...")
    for review in tqdm(reviews_subset):
        entities = get_entities(nlp, review)

        # add this review as a count to an entity
        for ent in entities:
            entities_with_count[ent.lower()] += 1

        review_entities.append(entities)
        
    # only grab entities that have enough reviews
    print("Filtering entities to have enough reviews...")
    entities_with_enough_reviews = []
    threshold = 30
    for key, value in entities_with_count.items():
        if value >= threshold:
            entities_with_enough_reviews.append(key)
            
    # TRUE RANKINGS CALCULATION
    # for each entity, average ratings
    true_rankings = defaultdict(list)

    print("Calculating Yelp Star Rankings... ")
    for entity in entities_with_enough_reviews:
        true_rankings['entity'] += [entity]
        entity_reviews = subset[subset.text.str.contains(entity, case=False)]
        true_rankings['average_stars'] += [np.mean(entity_reviews.stars)]

    true_rankings = pd.DataFrame(true_rankings)
    
    # PREDICTION RANKING CALCULATION
    print("Calculating Prediction Rankings...")
    # Filter entities of each review to be from the entities_with_enough_review set
    entity_filter = set(entities_with_enough_reviews)

    filtered_entities = []

    for entities in review_entities:
        filtered = []
        for ent in entities:
            ent = ent.lower()
            if ent in entity_filter:
                filtered.append(ent)
        filtered_entities.append(filtered)
    
#     # run validation for each rule
#     for rule in rules:

    # perform sentiment analysis for each review with filtered entities above
    predicted_scores = defaultdict(list)

    print("Performing sentiment analysis for each review... ")
    for i, review in enumerate(tqdm(reviews_subset)):
        entities = filtered_entities[i]

    #     print(review)

        scores = werk(review, entities, parser = parser, sentiment_package='vader', rule=rule)

        # save results 
        for entity, score in scores:
            predicted_scores[entity] += [score]

    # create rankings from scores
    predicted_rankings = defaultdict(list)
    for entity, scores in predicted_scores.items():
        predicted_rankings['entity'] += [entity]
        predicted_rankings['predicted_score'] += [np.mean(scores)]

    predicted_rankings = pd.DataFrame(predicted_rankings)

    #### may not be necessary to do these castings
    predicted_rankings['entity'] = predicted_rankings['entity'].astype(str)
    true_rankings['entity'] = true_rankings['entity'].astype(str)
    ####
    
    full_rankings = true_rankings.merge(predicted_rankings, how='left').fillna(0)

    # spearman correlation metric
    print("Rankings result: ")
    print(full_rankings)
    
    corr, pvalue = spearmanr(full_rankings.average_stars, full_rankings.predicted_score)
    print("Spearman Correlation Score: ", corr)
    correlation_scores.append(corr)
        #     print(werk(review, entities, parser = parser,sentiment_package='vader'))

RULE:  rule_3


Running on restaurant  HhVmDybpU7L50Kb5A0jXTg ...
Number of Reviews left after subset length:  1600
Extracting entities from each review...



Filtering entities to have enough reviews...
Calculating Yelp Star Rankings... 
Calculating Prediction Rankings...
Performing sentiment analysis for each review... 


KeyboardInterrupt: 

In [ ]:
print("Final Correlation Score: ", np.mean(correlation_scores))